In [1]:
import pandas as pd

import requests  

import re
import math
import random
import time

from pymongo import MongoClient

In [2]:
# 设置请求头
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Host': 'www.woshipm.com',
    'Referer': 'http://www.woshipm.com/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36'
}

In [3]:
# 创建连接
client = MongoClient('localhost', 27017)
# 获取数据库
db = client['iSpider']
# 获取集合
articles = db.woshipm_article
content = db.woshipm_article_content

In [4]:
# 获取 JSON 函数
def getJSON(url, headers):
    """ Get JSON
    @ param url: str 
    @ param headers: dict, request headers
    @ return json: JSON, result
    """
    res = requests.get(url, headers=headers) 
    res.raise_for_status()  #抛出异常
    res.encoding = 'utf-8'  
    json = res.json()
    return json

In [5]:
# 获取文章列表函数
def getArticleList(page, collection, headers):
    """ Get Article List
    @ param page: int
    @ param headers: dict, request headers
    @ param collection: pymongo.collection.Collection
    """
    url = 'http://www.woshipm.com/__api/v1/stream-list/page/{0}'.format(page)
    json = getJSON(url, headers)
    
    if json['success'] == 'true':
        for article in json['payload']:
            collection.insert_one(article)
    else:
        print(json['success'])
        
    print("获取第 {0} 页成功！".format(page))

In [6]:
article_list = articles.find({'is_event': False})

In [7]:
df_list = []

for article in article_list:
    df_row = []
    df_row.append(article['id'])
    df_row.append(article['permalink'])
    df_list.append(df_row)
    
df = pd.DataFrame(df_list)
df.head()

,0,1
0,1797322,http://www.woshipm.com/pmd/1797322.html
1,1798253,http://www.woshipm.com/evaluating/1798253.html
2,1799172,http://www.woshipm.com/operate/1799172.html
3,1798571,http://www.woshipm.com/it/1798571.html
4,1795015,http://www.woshipm.com/zhichang/1795015.html


In [8]:
df.to_csv('target.csv', index=False)